In [ ]:
import numpy as np
from autodm.llm import get_llm

from pydantic import BaseModel, Field
from typing import Union, Literal, Optional, Any, List
from rich import print
import matplotlib.pyplot as plt

In [ ]:
from llama_index.core.program import LLMTextCompletionProgram

In [ ]:
llm = get_llm()

In [ ]:
scales = Literal['miles', 'feet']

In [ ]:
scales = Literal['miles', 'feet']

class GridWorld(BaseModel):
    width:int = Field(50, description="Width of the grid", ge=1)
    height:int = Field(50, description="Height of the grid", ge=1)
    scale:int = Field(1, description="Number of units ", ge=1)
    scale_unit: scales = Field('miles', description="Unit of the scale")
    grid: Optional[Any] = Field(None, description="Grid of the world")

    def model_post_init(self, __context: Any) -> None:
        self.grid = np.zeros((self.height, self.width), dtype=int)

    def show(self):
        plt.imshow(self.grid, cmap='hot')

    def __str__(self):
        s = ""
        for i in world.grid:
            s += str(i.tolist())
            s += '\n'
        return s
    def __repr__(self):
        return self.__str__()

In [ ]:
REGIONAL_GRID_VALUES = {
    0: "wilderness",
    1: "city/town",
    2: "road",
    3: "water",
    4: "foothills",
    5: "mountain",
}

class MapItem(BaseModel):
    xs: List[int]
    ys: List[int]
    value: int = Field(0, description="Value to be added to the grid")

    def add_to_map(self, grid: GridWorld):
        grid.grid[self.ys, self.xs] = self.value

In [ ]:
world = GridWorld()

In [ ]:
region_s = "\n".join([f"{k}: {v}" for k, v in REGIONAL_GRID_VALUES.items()])
prompt = f"""\
You are a creative dungeon master filling in a D&D map with one of the following categories:
{region_s}
Here is the current grid:
{str(world)}
Add one river or other body of water to the world. \
If it's a river, it should touch at least 2 sides of the map. \
All bodies of water should be connected. \
Please follow the requested json format.

Answer: \
"""

In [ ]:
region_s = "\n".join([f"{k}: {v}" for k, v in REGIONAL_GRID_VALUES.items()])
prompt = f"""\
You are a creative dungeon master filling in a D&D map with one of the following categories:
{region_s}
Here is the current {world.width} x {world.height} grid:
{str(world)}
Please add the following to the map:
2-3 cities/towns
Meandering roads connecting the cities
A river that touches at least 2 sides of the map
Mountains and foothills

Answer: \
"""

In [ ]:
output = get_llm().complete(prompt)

In [ ]:
output

In [ ]:
program = LLMTextCompletionProgram.from_defaults(output_cls=MapItem, prompt_template_str=prompt, llm=get_llm())

In [ ]:
output

In [ ]:
output.add_to_map(world)

In [ ]:
world.show()